In [ ]:
# Libraries + data loading

import numpy as np
import torch
import math
import time
from skimage import io, filters, color
from pathlib import Path
import matplotlib.pyplot as plt
from scipy.fft import fft2, ifft2
from scipy.signal import correlate
import scipy


sin = math.sin
cos = math.cos
pi = math.pi



path = Path(__file__).resolve()    # Path of the folder where this file is located

image_name = 'image'
image_format = 'jpg'

image = io.imread(path.parent / 'data' / image_name + '.' + image_format)  # Load image 

def view_image(img,dpi=300,cmap='gray'):
    plt.figure(dpi=dpi)
    if cmap is None:
        plt.imshow(img)
    else:
        plt.imshow(img,cmap=cmap)
    plt.axis('off')
    plt.show()


def zero_pad(I: np.ndarray, pad_width: int = 5) -> np.ndarray:
    """
    Pad the image I with zeros on all sides.
    """
    return np.pad(I, pad_width, mode='constant', constant_values=0)


In [ ]:
# Implementing deblurring method from article 

def kernel_estimation(I: np.ndarray, K: np.ndarray, alph: float, N_iter = None) -> np.ndarray:
    """
    Estimate the blur kernel using the power spectrum of the image's autocorrelation of the shear projection power spectrum, then retrieving the phase 
    and finally estimating the true support of the kernel so that next step of the loop will be more accurate.
    """
    p = K.shape[0]

    A = ComputeProjectionAngleSet(p)  
    R = ComputeProjectionsAutocorrelation(I, A, p, alph)
    S = initial_support_estimation(R, A)  

    for _ in range(N_iter):
        
        # Step 1: Compute the power spectrum
        H = estimate_power_spectrum(R, S)

        # Step 2: Retrieve the phase using the method of [1]
        K = retrieve_phase(H, p)

        # Step 3: Estimate the true support of the kernel
        S = estimate_true_support(K, A)


    

def ComputeProjectionAngleSet(p: int, r = 4) -> np.ndarray:
    P = (r*p)//2
    values = range(-P, P+1)
    slopes = set()

    for i in values:
        if i == 0:
            continue

        for j in values:
            g = math.gcd(i,j) if j!=0 else abs(i)
            i_, j_ = i//g, j//g

            if i_ < 0:
                i_, j_ = -i_, -j_

            slopes.add((i_, j_))
    
    angles = []
    for i_, j_ in slopes: 
        slope = j_/i_
        theta = math.atan(slope)

        if np.isclose(abs(theta), pi/2):
            theta = np.sign(theta)*pi/2

        angles.append(theta)
    
    return sorted(angles)


def ComputeProjectionsAutocorrelation(I: np.ndarray, A: np.ndarray, p: int, alph: float) -> np.ndarray:

    vx, vy = derivatives(I)
    AC = dict()

    for theta in A:
        q = shear_projection(vx, vy, theta)
        ac = autocorrelation(q, p)
        h = compensation_filter(len(ac), alph) # Although len(ac) is supposed to be p ??
        ac_deconv = deconvolve(ac, h, alph)
        AC[theta] = ac_deconv

    return AC


def initial_support_estimation(AC: dict, A: np.ndarray, slope: float = 2/70) -> np.ndarray:

    S_prime = [np.argmin(AC[theta]) for theta in A]
    S_theta = [len(AC[0]) for theta in A]

    for i, theta_i in enumerate(A):
        if S_prime[i] < S_theta[i]:
            S_theta[i] = S_prime[i]
            for j, theta_j in enumerate(A):
                S_theta[j] =  min(S_theta[j], S_prime[i] + slope*abs(i-j))

    return np.array(S_theta, dtype=np.int32)


def estimate_power_spectrum(AC: dict, S: np.ndarray, A: np.ndarray) -> np.ndarray:
    
    RP = AC.copy()

    for i, theta in enumerate(A):
        mu = AC[theta][S[i]]

        for k in range(len(AC[theta])):  
            if k-len(AC[theta])//2 >= -S[i] or k-len(AC[theta])//2 <= S[i]:
                RP[theta][k] = max(AC[theta][k] - mu, 0)

            else:
                RP[theta][k] = 0

        RP[theta]= RP[theta]/np.sum(RP[theta])

    H_mod = 
    return RP

        


def retrieve_phase(H: np.ndarray, p: int) -> np.ndarray:
    pass

def estimate_true_support(K: np.ndarray, A: np.ndarray) -> np.ndarray:
    pass



# Utility functions for kernel estimation

def convolve2D(I: np.ndarray, K: np.ndarray) -> np.ndarray:
    """
    Perform a linear 2D convolution of image I with kernel K. We first apply zero-padding to I according to the article
    """
    p = K.shape[0]
    pad_width = p//2
    I_padded = zero_pad(I, pad_width)
    J = np.zeros_like(I, dtype = np.float32)
    H, W = I.shape[0], I.shape[1]
    for i in range(H):
        for j in range(W):
            J[i,j] = np.sum(I_padded[i:i+p, j:j+p] * K)
    return J

def derivatives(v: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    """
    Compute the horizontal and vertical derivatives of image I using Goldstein filter.
    """
    d = np.asarray([3,-32,168,-672,0,672,-168,32,-3])/840 # Goldstein filter
    d_h = d.reshape(1, -1)
    d_v = d.reshape(-1, 1)
    v_x = scipy.signal.convolve2d(v, d_h, mode='same', boundary='fill', fillvalue=0)
    v_y = scipy.signal.convolve2d(v, d_v, mode='same', boundary='fill', fillvalue=0)
    return v_x, v_y

def shear_projection(vx: np.ndarray, vy: np.ndarray, theta: float) -> np.ndarray:
    """
    Compute the shear projection of image I at angle theta using the Fourier slice theorem.
    """
    H, W = vx.shape
    q = np.zeros(H+W, dtype= np.float32) # Projection array of siz H+W because extreme cases for offest are -W/2 - H/2 and W/2 + H/2

    for i in range(H):
        for j in range(W):
            x, y = i - (H-1)/2, j - (W-1)/2

            if abs(theta) <= pi/4:      # horizontal projection
                offset = int(np.round(x + y * math.tan(theta)))
            else:                       # vertical projection   
                offset = int(np.round(y + x / math.tan(theta)))
                             
            w = vx[i,j]*cos(theta) + vy[i,j]*sin(theta)
            q[offset + (H+W)//2] += w   # Nearest neighbor accumulation

    return q


def autocorrelation(q: np.ndarray, max_lag: int) -> np.ndarray:
    ac = correlate(q, q, mode='full')
    center = len(ac) // 2
    return ac[center-max_lag:center+max_lag+1]


def compensation_filter(p: np.ndarray, alph: float) -> np.ndarray:
    """
    Return a compensation filter that will be applied to the autocorrelation of the shear projections.
    This is a sharpening filter to compensate for camera intrinsic blur.
    """
    values = range(-p, p+1)
    Z = sum((1+abs(k))**(-alph) for k in values)
    H = np.array([(1+abs(k))**(-alph)/Z for k in values])

    return H


def conjugate_gradient(apply_A, b, x0=None, max_iter=200, tol=1e-6):
    """
    Conjugate Gradient pour résoudre A x = b, où apply_A est l'opérateur A.
    """
    x = np.zeros_like(b) if x0 is None else x0.copy()
    r = b - apply_A(x)
    p = r.copy()
    rs_old = np.dot(r, r)

    if rs_old == 0:
        return x

    for _ in range(max_iter):
        Ap = apply_A(p)
        denom = np.dot(p, Ap)
        if denom == 0:
            break
        alpha = rs_old / denom
        x += alpha * p
        r -= alpha * Ap
        rs_new = np.dot(r, r)
        if np.sqrt(rs_new) < tol * np.linalg.norm(b):
            break
        p = r + (rs_new / rs_old) * p
        rs_old = rs_new
    return x


def conv1d_replicate(x: np.ndarray, h: np.ndarray) -> np.ndarray:
    """ 1D Convolution y = h * x with edge 'replicate'."""
    r = (len(h) - 1) // 2
    xpad = np.pad(x, (r, r), mode='edge')
    return np.convolve(xpad, h, mode='valid')


def deconvolve(ac: np.ndarray, h: np.ndarray, alpha: float, max_iter: int = 200, tol: float = 1e-6) -> np.ndarray:
    """ """
    n = len(ac)
    p = (n - 1) // 2          
    apply_A = lambda x: conv1d_replicate(x, h)
    x = conjugate_gradient(apply_A, ac, max_iter=max_iter, tol=tol)
    c = n // 2

    if np.any(x[max(0, c-2):min(n, c+3)] < 0):
        return ac
    
    return x

In [ ]:
# Deconvolution via TV minimization using split-Bregman method

# 1. first we deal with edge artifcats due to periodic boundary conditions

def fourier_kernel(kernel: np.ndarray, shape: tuple):
    """
    Computes the Fourier transform of the kernel and pads it to the desired shape.
    We use padding so that the kernel FT has the same shape as the image FT but the kernel is set on (0,0).
    """

    pad = [(0, s - k) for s, k in zip(shape, kernel.shape)]
    kernel_padded = np.pad(kernel, pad, mode='constant')
    kernel_ft = np.fft.fft2(np.fft.ifftshift(kernel_padded))
    return kernel_ft


def edge_tappering(I: np.ndarray, K: np.ndarray):
    """
    Apply edge tappering by blurring the image I with kernel K and adding this image J to I so that with smooth the edge.
    """

    I = I.astype(np.float32, copy=False) # converts it in float32 to avoid truncature problem 

    kh, kw = K.shape
    Ih, Iw = I.shape

    # Creating weighting mask so that we add J only on the edge (we only want to smooth the edge for periodic purposes)
    wx = np.ones((Ih, Iw), dtype=np.float32)
    wy = np.ones((Ih, Iw), dtype=np.float32)

    X, Y = np.meshgrid(np.arange(Iw, dtype=np.float32), np.arange(Ih, dtype=np.float32))    # X and Y are coordinates grid for the mask

    # Smoothing the transition from 0 (edge) to 1 (interior of the weighting window) applying sin**2 transformation 
    if kh > 1:
        wy[:kh, :]   = sin(Y[:kh, :] * pi / (2*kh - 1))**2
        wy[-kh:, :]  = np.sin((Ih - 1 - Y[-kh:, :]) * pi / (2*kh - 1))**2

    if kw > 1:
        wx[:, :kw]   = sin(X[:, :kw] * pi / (2*kw - 1))**2
        wx[:, -kw:]  = sin((Iw - 1 - X[:, -kw:]) * pi / (2*kw - 1))**2

    w = wx * wy     # full mask
    fK = fourier_kernel(K, I.shape)
    J = np.real(ifft2(fft2(I)*fK)).astyp(np.float32)

    return J*(1.0 - w) + I*w     # if w[i,j] = 1, we keep I[i,j], if w[i,j] = 0, we keep J[i,j] (typically on the edges), otherwise we combine both




# Utils functions for next part


def conv(im: np.ndarray, K: np.ndarray, fourier_form: bool = False) -> np.ndarray:
    """
    Circular convolution between an image and a kernel using FFT.
    """
    if not fourier_form:
        # Expect spatial im and spatial kernel K.
        # Fourier_kernel must pad & circularly center K to im.shape before FFT.
        Kf = fourier_kernel(K, im.shape)  # assumes this utility exists in your codebase
        Im = fft2(im)
        return np.real(ifft2(Im * Kf))
    else:
        # Expect both inputs already in Fourier domain: im -> Im, K -> Kf
        return np.real(ifft2(im * Kf))

def grad_circular(u: np.ndarray) -> np.ndarray:
    """
    Circular (periodic) forward gradient of a 2D image.
    """
    # Horizontal forward difference with wrap around on the last column
    gx = np.c_[(u[:, 0] - u[:, -1]).reshape(-1, 1), u[:, 1:] - u[:, :-1]]
    # Vertical forward difference with wrap around on the last row
    gy = np.r_[(u[0, :] - u[-1, :]).reshape(1, -1), u[1:, :] - u[:-1, :]]
    return np.stack((gx, gy), axis=0)

def div_circular(c: np.ndarray) -> np.ndarray:
    """
    Circular (periodic) divergence, adjoint of -grad_circular.
    """
    cx, cy = c[0], c[1]

    # Backward difference in x with wrap
    dx = np.c_[cx[:, 1:] - cx[:, :-1], (cx[:, 0] - cx[:, -1]).reshape(-1, 1)]
    # Backward difference in y with wrap
    dy = np.r_[cy[1:, :] - cy[:-1, :], (cy[0, :] - cy[-1, :]).reshape(1, -1)]

    return dx + dy


def norm_l2(X: np.ndarray) -> float:
    return float(np.linalg.norm(X.ravel(), ord=2))





def TV_deconvolution(f: np.ndarray, K: np.ndarray, lam: float, mu: float, n_iter: int) -> np.ndarray:
    """
    Perform TV deconvolution using the split-Bregman method.
    """
    # Initialize variables
    u = f.copy()
    d = np.zeros((2, *f.shape), dtype=f.dtype)
    b = np.zeros((2, *f.shape), dtype=f.dtype)

    for _ in range(n_iter):
        # Solve d-problem
        d = d_problem(u, b, lam, mu)
        
        # Solve u-problem
        u = u_problem(f, K, d, b, mu)
        
        # Update Bregman variable
        b += grad_circular(u) - d

    return u



# 2. Solving TV minimization problem using split-bregmann method (basically we split the problem using a new variable d)


# 2.a   u_problem

def u_problem(f: np.ndarray, K: np.ndarray, d: np.ndarray, b: np.ndarray, lamb: float,  mu: float) -> np.ndarray:
    """
    Solve the u-problem in the split-Bregman method for TV minimization.
    """
    # Fourier transforms
    f_ft = fft2(f)
    K_ft = fourier_kernel(K, f.shape)
    K_conj_ft = np.conj(K_ft)

    # Compute the divergence of (d - b)
    div_db = div_circular(d - b)
    div_db_ft = fft2(div_db)

    den = lamb*np.abs(K_ft)**2 + mu * (4 - 2 * (np.cos(2 * np.pi * np.arange(f.shape[1]) / f.shape[1])[:, None] + np.cos(2 * np.pi * np.arange(f.shape[0]) / f.shape[0])[None, :]))
    num = lamb*K_conj_ft * f_ft + mu * div_db_ft

    # Solve for u in the frequency domain and transform back to spatial domain
    u_ft = num / den
    u = np.real(ifft2(u_ft))
    
    return u



# 2.b   d_problem 

def d_problem(u: np.ndarray, b: np.ndarray, lam: float, mu: float) -> np.ndarray:
    """
    Solve the d-problem in the split-Bregman method for TV minimization.
    """
    # Compute the gradient of u
    grad_u = grad_circular(u)
    
    # Update d using the shrinkage operator
    d = grad_u + b
    norm_d = np.sqrt(d[0]**2 + d[1]**2)
    shrink = np.maximum(0, norm_d - lam/mu) / (norm_d + 1e-10)  # Avoid division by zero
    d[0] *= shrink
    d[1] *= shrink
    
    return d




# TV solver

def TV_deblur(I: np.ndarray, K: np.ndarray, lam: float, mu: float, n_iter: int) -> np.ndarray:
    """
    Perform TV deconvolution using the split-Bregman method.
    """
    # Initialize variables
    u = I.copy()
    d = np.zeros((2, *I.shape), dtype=I.dtype)
    b = np.zeros((2, *I.shape), dtype=I.dtype)

    for _ in range(n_iter):

        # Solve u-problem
        u = u_problem(I, K, d, b, mu)

        # Solve d-problem
        d = d_problem(u, b, lam, mu)
        
        # Update Bregman variable
        b += grad_circular(u) - d

    return u
